In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [2]:
def gbm(n_steps=255, n_scenarios=1000, mu=0.07, sigma=0.15, s_0=100.0, prices=True):
    # Calculate the time step size (dt) as a fraction of the total period
    dt = 1 / n_steps
    # Add an extra step to include the initial value
    n_steps = int(n_steps) + 1
    
    # Generate random returns using a normal distribution
    # loc = (1 + mu)**dt is the mean, scale = (sigma * sqrt(dt)) is the standard deviation
    rets_plus_1 = np.random.normal(loc=(1 + mu)**dt, scale=(sigma * np.sqrt(dt)), size=(n_steps, n_scenarios))
    
    # Set the first row (initial returns) to 1 since no returns are accumulated initially
    rets_plus_1[0] = 1
    
    # Calculate cumulative product of returns to get the price paths
    prices = s_0 * pd.DataFrame(rets_plus_1).cumprod() if prices else rets_plus_1 - 1
    return prices

def cppi(gbm_sim, floor=80, multiplier=3, risk_free_rate=0.02, n_rebalance=10):
    # Get the number of time steps from the simulation data
    n_steps = gbm_sim.index.size

    # Calculate the steps between rebalances
    # If only one rebalance is required, do it halfway through the period
    if n_rebalance == 1:
        steps_per_rebalance = int(n_steps / 2)
    else:
        # Otherwise, calculate steps per rebalance based on the specified number of rebalances per year
        steps_per_rebalance = max(1, int(n_steps / n_rebalance))

    # Initialize DataFrames to store account value, cushion, and risky weights
    account_value = pd.DataFrame(index=gbm_sim.index, columns=gbm_sim.columns)
    cushion = pd.DataFrame(index=gbm_sim.index, columns=gbm_sim.columns)
    risky_weight = pd.DataFrame(index=gbm_sim.index, columns=gbm_sim.columns)
    
    # Set the initial account value to the initial price from the GBM simulation
    account_value.iloc[0] = gbm_sim.iloc[0]
    
    # Loop through each time step to update the portfolio
    for step in range(1, n_steps):
        # Check if it's time to rebalance or if it's the first step
        if step % steps_per_rebalance == 0 or step == 1:
            # Calculate the cushion (difference between account value and the floor)
            cushion.iloc[step - 1] = np.maximum(account_value.iloc[step - 1] - floor, 0)
            
            # Determine the proportion to invest in the risky asset based on the cushion
            risky_weight.iloc[step - 1] = multiplier * cushion.iloc[step - 1] / account_value.iloc[step - 1]
            
            # Ensure the risky weight is between 0 and 1
            risky_weight.iloc[step - 1] = np.clip(risky_weight.iloc[step - 1], 0, 1)
        else:
            # If not rebalancing, carry forward the previous weight
            risky_weight.iloc[step - 1] = risky_weight.iloc[step - 2] if step > 1 else 0
        
        # Calculate the safe asset weight as the complement of the risky weight
        safe_weight = 1 - risky_weight.iloc[step - 1]
        
        # Update the account value based on returns from risky and safe assets
        account_value.iloc[step] = (
            account_value.iloc[step - 1] * 
            (
                risky_weight.iloc[step - 1] * (gbm_sim.iloc[step] / gbm_sim.iloc[step - 1]) + 
                safe_weight * np.exp(risk_free_rate / 360)
            )
        )
    
    return account_value


In [18]:
# Interactive widgets
n_steps_widget = widgets.IntSlider(value=90, min=25, max=255, step=1, description='Steps')
n_scenarios_widget = widgets.IntSlider(value=100, min=10, max=1000, step=10, description='Scenarios')
mu_widget = widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01, description='Mu')
sigma_widget = widgets.FloatSlider(value=0.20, min=0.0, max=0.5, step=0.01, description='Sigma')

floor_widget = widgets.FloatSlider(value=90, min=50, max=100, step=1, description='Floor')
multiplier_widget = widgets.IntSlider(value=3, min=1, max=10, step=1, description='Multiplier')
rf_rate_widget = widgets.FloatSlider(value=0.05, min=0.0, max=0.1, step=0.01, description='Risk-free')
n_rebalance_widget = widgets.IntSlider(value=12, min=1, max=255, step=1, description='Rebalance')

# Button to run the simulation
run_button = widgets.Button(description="Run Simulation", button_style='success')

# Output widget to display results
output = widgets.Output()

# Function to run the simulation and plot results
def run_simulation(event):
    with output:
        output.clear_output()
        
        # Récupérer les valeurs des widgets
        n_steps = n_steps_widget.value
        n_scenarios = n_scenarios_widget.value
        mu = mu_widget.value
        sigma = sigma_widget.value
        s_0 = 100
        
        floor = floor_widget.value
        multiplier = multiplier_widget.value
        rf_rate = rf_rate_widget.value
        n_rebalance = n_rebalance_widget.value
        
        # Exécuter la simulation GBM
        gbm_sim = gbm(
            n_steps=n_steps, 
            n_scenarios=n_scenarios, 
            mu=mu, 
            sigma=sigma, 
            s_0=s_0,
            prices=True
        )
        
        # Appliquer la stratégie CPPI
        portfolio_values = cppi(
            gbm_sim, 
            floor=floor, 
            multiplier=multiplier, 
            risk_free_rate=rf_rate, 
            n_rebalance=n_rebalance
        )

        # Calculer la moyenne et l'écart-type du portefeuille CPPI
        cppi_mean = portfolio_values.mean(axis=1)
        cppi_std = portfolio_values.std(axis=1)
        
        # Calculer 2 fois l'écart-type au-dessus et en-dessous de la moyenne
        upper_bound = cppi_mean + cppi_std
        lower_bound = cppi_mean - cppi_std
        
        # Calculer la moyenne du portefeuille CPPI
        cppi_mean = portfolio_values.mean(axis=1)
        
        # Tracer les résultats
        plt.figure(figsize=(14, 7))
        
        # Tracer chaque simulation en gris
        plt.plot(portfolio_values, color='grey', linewidth=0.5, alpha=0.25)
        
        # Tracer la ligne du floor
        plt.axhline(y=floor, linestyle='--', color='red', label='Floor')
        
        # Tracer la moyenne du portefeuille CPPI
        plt.plot(cppi_mean, color='blue', alpha=0.75, label='CPPI Portfolio (Mean)')

        # Tracer 2 fois l'écart-type autour de la moyenne
        plt.plot(upper_bound, linestyle='--', linewidth=0.75, color='blue', label='Mean + Std Dev')
        plt.plot(lower_bound, linestyle='--', linewidth=0.75, color='blue', label='Mean - Std Dev')
        
        # Personnaliser le graphique
        #plt.yscale('log')
        plt.title("CPPI Strategy Simulation")
        plt.xlabel("Time Steps")
        plt.ylabel("Portfolio Value")
        plt.grid(True)
        plt.legend()
        plt.show()

# Bind the button to the function
run_button.on_click(run_simulation)

# Display the widgets
display(
    n_steps_widget, 
    n_scenarios_widget, 
    mu_widget, 
    sigma_widget, 
    floor_widget, 
    multiplier_widget, 
    rf_rate_widget, 
    n_rebalance_widget,
    run_button, output
)

IntSlider(value=90, description='Steps', max=255, min=25)

IntSlider(value=100, description='Scenarios', max=1000, min=10, step=10)

FloatSlider(value=0.05, description='Mu', max=0.2, step=0.01)

FloatSlider(value=0.2, description='Sigma', max=0.5, step=0.01)

FloatSlider(value=90.0, description='Floor', min=50.0, step=1.0)

IntSlider(value=3, description='Multiplier', max=10, min=1)

FloatSlider(value=0.05, description='Risk-free', max=0.1, step=0.01)

IntSlider(value=12, description='Rebalance', max=255, min=1)

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Output()

In [4]:
def avg_return(history: pd.Series, timeperiod: int = 255) -> float:
    returns = history.pct_change().dropna()
    return (1 + returns.sum()) ** (timeperiod/returns.count()) - 1

def avg_volatility(history: pd.Series, timeperiod: int = 255) -> float:
    returns = history.pct_change()
    return returns.std() * np.sqrt(timeperiod)

def max_drawdown(history: pd.Series) -> float:
    index = 100 * (1 + history.pct_change()).cumprod()
    peaks = index.cummax()
    drawdowns = (index - peaks) / peaks
    return drawdowns.min()

# étude du rendement et de la volatilité moyenne annuelle du S1P 500 de 2000 à nos jours
import yfinance as yf
history = yf.Ticker('^GSPC').history('max')['Close'].tz_localize(None).dropna()
history = history.loc['01/01/2000':'01/11/2024']

print(f'Avg annual return: {"{:.3}%".format(avg_return(history)*100)}')
print(f'Avg annual volatility: {"{:.3}%".format(avg_volatility(history)*100)}')

Avg annual return: 4.2%
Avg annual volatility: 19.7%


In [21]:
data_df = cppi(
    gbm(
        n_steps=90, 
        n_scenarios=1000, 
        mu=avg_return(history), 
        sigma=avg_volatility(history),
        s_0=100, 
        prices=True
    ),
    floor=90, 
    multiplier=3, 
    risk_free_rate=0.0475, 
    n_rebalance=12
)

In [22]:
result_df = pd.DataFrame()
result_df['Max drawdown'] = max_drawdown(data_df).describe().round(4)
result_df['Average return'] = avg_return(data_df, timeperiod=255).describe().round(4)
result_df['Average volatility'] = avg_volatility(data_df, timeperiod=255).describe().round(4)
result_df

,Max drawdown,Average return,Average volatility
count,1000.0000,1000.0000,1000.0000
mean,-0.0629,0.0847,0.1093
std,0.0211,0.2452,0.0326
min,-0.1679,-0.2231,0.0452
25%,-0.0742,-0.0777,0.0846
50%,-0.0599,0.0165,0.1038
75%,-0.0482,0.1700,0.1281
max,-0.0216,1.9591,0.2610
